**Use of Grid Search with Random Forest on the Boston Housing data**

**AWS Configuration required**

Fit2ec2 use your credentials stored in ~/.aws/credential

Make sure you have install aws-cli and it's configured

https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import the usual suspects
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
# fit2ec2
import fit2ec2 as fe

**Load and Split data**

In [ ]:
data = pd.read_csv('data/data.csv')

y = np.log(data.SalePrice)
X = data.drop(['SalePrice', 'Id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
                          X, y, random_state=42, test_size=.33)

print(X_train.shape)

**Initialize the Random Forest and the Grid Search**

In [ ]:
# Initialize Random Forest
rf = RandomForestRegressor()

In [ ]:
# Choose parameters for Grid Search
rf_params = {'n_estimators': range(150, 300, 10), 'max_depth': range(3, 15, 1),'min_samples_leaf':[0.04,0.06,0.08], 'max_features':[0.2,0.4,0.6,0.8]}

In [ ]:
# Don't forget the n_jobs parameters on a multi-cpu machine!
grid = GridSearchCV(estimator = rf,
                         param_grid = rf_params,
                         cv=2,
                         scoring='neg_mean_squared_error',
                         verbose=1,
                         n_jobs=-1)

**Initialize Fit2ec2**

In [ ]:
cmp = fe.Compute()

# Or specify the private key name
# cmpC = fe.Fit2ec2(keyname="mykey")

**Create the ec2 instance**

In [ ]:
# Create an EC2 with the default AWS Linux AMI and a type of t2.micro
cmp.create()

# Or specify the image id (AMI ID) and the instance type (see https://aws.amazon.com/ec2/instance-types/)
#cmp.create(imageId='ami-06ce3edf0cff21f07', instanceType='t2.micro')

**Fit the model with the data on the ec2 instance**

In [ ]:
grid = cmp.fit(grid, X_train, y_train)
# If you get NoValidConnectionsError, better to wait 30secs for the ec2 instance to turn on

In [ ]:
grid.best_score_

In [ ]:
grid.best_estimator_

**Terminate the ec2 instance and remove the private key**

In [ ]:
cmp.terminate()